# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Perform an ETL process with multiple tables and create a single dataset

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this SQL query to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
```

In [1]:
import sqlite3
import pandas as pd

# Create a connection
con = sqlite3.connect('lego.db')
# Create a cursor
cur = con.cursor()
# Select some data
cur.execute("""SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.head()

(4, 1)


,name
0,product_details
1,product_info
2,product_pricing
3,product_reviews


In [3]:
#preview of product_details

cur.execute('''
            SELECT * from product_details;
            ''')

product_details_df = pd.DataFrame(cur.fetchall())
product_details_df.head()

,0,1,2,3
0,630,blah,blahblah,Classic
1,2304,blah,blahblah,DUPLO®
2,7280,blah,blahblah,City
3,7281,blah,blahblah,City
4,7499,blah,blahblah,City


In [4]:
cur.execute('''
            SELECT * from product_info;
            ''')

product_info_df = pd.DataFrame(cur.fetchall())
product_info_df.head()

,0,1,2,3
0,630,4+,1,Brick Separator
1,2304,1½-5,1,LEGO® DUPLO® Green Baseplate
2,7280,5-12,2,Straight & Crossroad Plates
3,7281,5-12,2,T-Junction & Curved Road Plates
4,7499,5-12,24,Flexible and Straight Tracks


In [5]:
cur.execute('''
            SELECT * from product_pricing;
            ''')

product_pricing_df = pd.DataFrame(cur.fetchall())
product_pricing_df.head()

,0,1,2
0,75823,US,29.99
1,75822,US,19.99
2,75821,US,12.99
3,21030,US,99.99
4,21035,US,79.99


In [6]:
cur.execute('''
            SELECT * from product_reviews;
            ''')

product_reviews_df = pd.DataFrame(cur.fetchall())
product_reviews_df.head()

,0,1,2,3,4,5
0,630,180.0,4.0,Very Easy,4.8,4.6
1,2304,15.0,4.4,Easy,4.0,3.3
2,7280,85.0,4.1,Very Easy,3.5,2.3
3,7281,40.0,4.1,Very Easy,3.9,2.8
4,7499,89.0,2.9,Easy,2.5,2.2


In [7]:
merged = pd.merge(product_reviews_df, product_pricing_df, product_details_df, product_info_df)

KeyError:            0     1     2                                                  3
0        630    4+     1                                    Brick Separator
1       2304  1½-5     1                       LEGO® DUPLO® Green Baseplate
2       7280  5-12     2                        Straight & Crossroad Plates
3       7281  5-12     2                    T-Junction & Curved Road Plates
4       7499  5-12    24                       Flexible and Straight Tracks
..       ...   ...   ...                                                ...
739   853638    7+    61  THE LEGO® BATMAN MOVIE Batman™ Minifigure Coll...
740  2000409    6+  4900                             Window Exploration Bag
741  2000414    6+   219                                        Starter Kit
742  2000430    6+  2631                         Identity and Landscape Kit
743  2000431    6+  2455                                    Connections Kit

[744 rows x 4 columns]

In [9]:
# okay, let's try it by figuring out how they join

cur.execute('''
            SELECT * from product_info
            JOIN product_details
                  USING(prod_id)
                  JOIN product_pricing
                  USING(prod_id)
                  JOIN product_reviews
                  USING(prod_id);
            ''')

combined_df = pd.DataFrame(cur.fetchall())
combined_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,75823,6-12,277,Bird Island Egg Heist,blah,blahblah,Angry Birds™,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,6-12,168,Piggy Plane Attack,blah,blahblah,Angry Birds™,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,6-12,74,Piggy Car Escape,blah,blahblah,Angry Birds™,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,12+,1032,United States Capitol Building,blah,blahblah,Architecture,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,12+,744,Solomon R. Guggenheim Museum®,blah,blahblah,Architecture,US,79.99,14.0,3.2,Challenging,4.6,4.1


In [11]:
combined_df.to_csv('Combined_Data.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity! In this lab, you successfully executed an ETL process to merge different tables!